In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
import seaborn as sns
import sklearn
import random
import math
import time


from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from tensorflow.keras import optimizers
from sklearn.decomposition import PCA

from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [2]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR', 'farB','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 120
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(61249, 27) (41214, 26) (248, 1)


In [3]:
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(LSTM(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    # model.save_weights(weights_file)

    return model

In [4]:
# X_cr_train, X_cr_test= StandardScaler().fit_transform(X_train_interim[remaining_sensors]), StandardScaler().fit_transform(X_test_interim[remaining_sensors])
# pca = PCA()

# component_train , component_test = pca.fit(X_cr_train).transform(X_cr_train), pca.transform(X_cr_test)
# # print(pca.explained_variance_, np_component) # choos component which lambda >1 # kaiser

# np_component = len(pca.explained_variance_[pca.explained_variance_>1])
# print(pca.explained_variance_,'\n', "Nb components: ", np_component) # choos component which lambda >1 # kaiser
# comp = ['comp' + str(i) for i in range(1,np_component+1)]
# X_train_interim[comp],  X_test_interim[comp]= component_train[:, :np_component], component_test[:, :np_component]


In [5]:
%%time
results_pca = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
    
    #PCA data reduction 
    X_cr_train, X_cr_test= StandardScaler().fit_transform(X_train_interim[remaining_sensors]), StandardScaler().fit_transform(X_test_interim[remaining_sensors])
    pca = PCA()
    component_train , component_test = pca.fit(X_cr_train).transform(X_cr_train), pca.transform(X_cr_test)
    # print(pca.explained_variance_, np_component) # choos component which lambda >1 # kaiser

    np_component = len(pca.explained_variance_[pca.explained_variance_>1])
    print(pca.explained_variance_,'\n', "Nb components: ", np_component) # choos component which lambda >1 # kaiser
    comp = ['comp' + str(i) for i in range(1,np_component+1)]
    X_train_interim[comp],  X_test_interim[comp]= component_train[:, :np_component], component_test[:, :np_component]
    

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length, comp)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,comp, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    
    input_shape = (sequence_length, len(comp))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_pca = pd.concat([results_pca, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_pca.to_csv('results/pca/fd004.csv')


[7.95914066e+00 5.20347330e+00 1.05754225e+00 9.73443909e-01
 6.01841369e-01 3.89076541e-01 1.95285565e-01 1.42521636e-01
 1.18388892e-01 9.96185263e-02 9.54720058e-02 7.55025638e-02
 3.73317439e-02 2.33308417e-02 1.38792003e-02 9.96708843e-03
 4.46146340e-03] 
 Nb components:  3
(51538, 40, 3) (51538, 1) (248, 40, 3)
Epoch 1/20

 460/1611 [=======>......................] - ETA: 14s - loss: 992.3143

KeyboardInterrupt: 

In [ ]:
results_pca

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,33.517714,0.0,20798.519974,0.0,1123.437134,0.0,[64],0.2,tanh,32,40,184.166592
1,30.657440,0.0,11337.076398,0.0,939.878723,0.0,[64],0.2,tanh,32,40,184.555245
2,27.935600,0.0,7179.744395,0.0,780.397766,0.0,[64],0.2,tanh,32,40,179.203301
3,33.057681,0.0,14701.550987,0.0,1092.810303,0.0,[64],0.2,tanh,32,40,180.289082
4,31.380157,0.0,11612.759988,0.0,984.714294,0.0,[64],0.2,tanh,32,40,184.788765


### FD002

In [ ]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [ ]:
%%time
results_pca002 = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.2
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [32]
    dropout = 0.1
    activation = 'tanh'
    batch_size = 128
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
    
    #PCA data reduction 
    X_cr_train, X_cr_test= StandardScaler().fit_transform(X_train_interim[remaining_sensors]), StandardScaler().fit_transform(X_test_interim[remaining_sensors])
    pca = PCA()
    component_train , component_test = pca.fit(X_cr_train).transform(X_cr_train), pca.transform(X_cr_test)
    # print(pca.explained_variance_, np_component) # choos component which lambda >1 # kaiser

    np_component = len(pca.explained_variance_[pca.explained_variance_>1])
    print(pca.explained_variance_,'\n', "Nb components: ", np_component) # choos component which lambda >1 # kaiser
    comp = ['comp' + str(i) for i in range(1,np_component+1)]
    X_train_interim[comp],  X_test_interim[comp]= component_train[:, :np_component], component_test[:, :np_component]
    

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length, comp)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,comp, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    
    input_shape = (sequence_length, len(comp))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_pca002 = pd.concat([results_pca002, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_pca002.to_csv('results/pca/fd002.csv')

[1.04271508e+01 2.66149542e+00 1.05555422e+00 4.67459514e-01
 2.81050559e-01 2.11386654e-01 1.92544371e-01 1.64287263e-01
 1.32272442e-01 1.16578816e-01 1.04679947e-01 7.46965656e-02
 4.85539287e-02 3.45854321e-02 1.97697647e-02 8.23190831e-03] 
 Nb components:  3
(43619, 40, 3) (43619, 1) (259, 40, 3)
Epoch 1/20
341/341 [==============================] - 3s 7ms/step - loss: 1384.3842 - val_loss: 876.1518
Epoch 2/20
341/341 [==============================] - 2s 6ms/step - loss: 433.4120 - val_loss: 823.0498
Epoch 3/20
341/341 [==============================] - 2s 6ms/step - loss: 424.6640 - val_loss: 789.3731
Epoch 4/20
341/341 [==============================] - 2s 6ms/step - loss: 387.0952 - val_loss: 873.8283
Epoch 5/20
341/341 [==============================] - 2s 6ms/step - loss: 300.4011 - val_loss: 745.8162
Epoch 6/20
341/341 [==============================] - 2s 6ms/step - loss: 258.9048 - val_loss: 634.3585
Epoch 7/20
341/341 [==============================] - 2s 6ms/step - los

In [ ]:
results_pca002

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,23.540787,0.0,3249.123771,0.0,554.168701,0.0,[32],0.1,tanh,128,40,44.072514
1,26.608995,0.0,10540.041642,0.0,708.038696,0.0,[32],0.1,tanh,128,40,48.091069
2,24.949041,0.0,4426.846722,0.0,622.454651,0.0,[32],0.1,tanh,128,40,50.391524
3,26.832192,0.0,4031.455326,0.0,719.966553,0.0,[32],0.1,tanh,128,40,49.096392
4,26.628517,0.0,8183.105448,0.0,709.077881,0.0,[32],0.1,tanh,128,40,48.870997


### FD003

In [ ]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD003.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(24720, 27) (16596, 26) (100, 1)


In [ ]:
%%time
results_pca003 = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.1
    sequence_length = 35
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
    
    #PCA data reduction 
    X_cr_train, X_cr_test= StandardScaler().fit_transform(X_train_interim[remaining_sensors]), StandardScaler().fit_transform(X_test_interim[remaining_sensors])
    pca = PCA()
    component_train , component_test = pca.fit(X_cr_train).transform(X_cr_train), pca.transform(X_cr_test)
    # print(pca.explained_variance_, np_component) # choos component which lambda >1 # kaiser

    np_component = len(pca.explained_variance_[pca.explained_variance_>1])
    print(pca.explained_variance_,'\n', "Nb components: ", np_component) # choos component which lambda >1 # kaiser
    comp = ['comp' + str(i) for i in range(1,np_component+1)]
    X_train_interim[comp],  X_test_interim[comp]= component_train[:, :np_component], component_test[:, :np_component]
    

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length, comp)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,comp, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    
    input_shape = (sequence_length, len(comp))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_pca003 = pd.concat([results_pca003, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_pca003.to_csv('results/pca/fd003.csv')

[8.19095667e+00 5.49399656e+00 8.81347307e-01 7.81564822e-01
 3.51238057e-01 1.43235420e-01 4.09913921e-02 3.53324081e-02
 2.44339253e-02 1.75337353e-02 1.33853528e-02 1.23424994e-02
 6.63828566e-03 3.52672765e-03 3.10180225e-03 1.02230366e-03] 
 Nb components:  2
(21320, 35, 2) (21320, 1) (100, 35, 2)
Epoch 1/20
667/667 [==============================] - 5s 6ms/step - loss: 836.8309 - val_loss: 944.6252
Epoch 2/20
667/667 [==============================] - 4s 6ms/step - loss: 289.8248 - val_loss: 1114.4414
Epoch 3/20
667/667 [==============================] - 3s 5ms/step - loss: 212.7500 - val_loss: 959.8607
Epoch 4/20
667/667 [==============================] - 4s 6ms/step - loss: 192.4150 - val_loss: 878.5651
Epoch 5/20
667/667 [==============================] - 4s 5ms/step - loss: 183.3982 - val_loss: 832.0654
Epoch 6/20
667/667 [==============================] - 4s 6ms/step - loss: 180.6049 - val_loss: 874.3455
Epoch 7/20
667/667 [==============================] - 4s 6ms/step - los

In [ ]:
results_pca003

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,27.169472,0.0,1606.809450,0.0,738.180176,0.0,[64],0.2,tanh,32,35,74.372573
1,27.634920,0.0,1340.860313,0.0,763.688843,0.0,[64],0.2,tanh,32,35,72.090802
2,26.560636,0.0,1262.115565,0.0,705.467346,0.0,[64],0.2,tanh,32,35,68.616933
3,29.689165,0.0,1721.257225,0.0,881.446533,0.0,[64],0.2,tanh,32,35,69.500571
4,27.079166,0.0,1642.169688,0.0,733.281250,0.0,[64],0.2,tanh,32,35,71.682696


### FD001

In [ ]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(20631, 27) (13096, 26) (100, 1)


In [ ]:
%%time
results_pca001 = pd.DataFrame()
for SEED in range(5):  
    set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.1
    sequence_length = 30
    epochs = 20
    nodes_per_layer = [128]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 64
    remaining_sensors = remaining_sensors    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
    
    #PCA data reduction 
    X_cr_train, X_cr_test= StandardScaler().fit_transform(X_train_interim[remaining_sensors]), StandardScaler().fit_transform(X_test_interim[remaining_sensors])
    pca = PCA()
    component_train , component_test = pca.fit(X_cr_train).transform(X_cr_train), pca.transform(X_cr_test)
    # print(pca.explained_variance_, np_component) # choos component which lambda >1 # kaiser

    np_component = len(pca.explained_variance_[pca.explained_variance_>1])
    print(pca.explained_variance_,'\n', "Nb components: ", np_component) # choos component which lambda >1 # kaiser
    comp = ['comp' + str(i) for i in range(1,np_component+1)]
    X_train_interim[comp],  X_test_interim[comp]= component_train[:, :np_component], component_test[:, :np_component]
    

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length, comp)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,comp, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    
    input_shape = (sequence_length, len(comp))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time': training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_pca001 = pd.concat([results_pca001, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_pca001.to_csv('results/pca/fd001.csv')

[1.16218316e+01 2.22745151e+00 7.53919107e-01 7.60689125e-02
 5.80733626e-02 4.69453854e-02 4.15078334e-02 3.47383080e-02
 3.32981565e-02 2.89969632e-02 2.53785317e-02 1.84836056e-02
 1.69423744e-02 1.35008525e-02 3.59058504e-03] 
 Nb components:  2
(17731, 30, 2) (17731, 1) (100, 30, 2)
Epoch 1/20
278/278 [==============================] - 6s 18ms/step - loss: 1091.9460 - val_loss: 993.8204
Epoch 2/20
278/278 [==============================] - 4s 13ms/step - loss: 224.6014 - val_loss: 880.8183
Epoch 3/20
278/278 [==============================] - 4s 13ms/step - loss: 192.0079 - val_loss: 687.6577
Epoch 4/20
278/278 [==============================] - 3s 12ms/step - loss: 187.4294 - val_loss: 767.1495
Epoch 5/20
278/278 [==============================] - 4s 14ms/step - loss: 174.4967 - val_loss: 792.8438
Epoch 6/20
278/278 [==============================] - 3s 12ms/step - loss: 170.6644 - val_loss: 663.6057
Epoch 7/20
278/278 [==============================] - 3s 10ms/step - loss: 172.4

In [ ]:
results_pca001

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,29.968799,0.0,2033.496475,0.0,898.128906,0.0,[128],0.2,tanh,64,30,64.759486
1,28.845945,0.0,1967.829243,0.0,832.088501,0.0,[128],0.2,tanh,64,30,60.036631
2,28.059698,0.0,1822.027067,0.0,787.346619,0.0,[128],0.2,tanh,64,30,59.399234
3,28.940213,0.0,1941.406954,0.0,837.535889,0.0,[128],0.2,tanh,64,30,63.134585
4,27.615634,0.0,1697.604402,0.0,762.623291,0.0,[128],0.2,tanh,64,30,61.557082
